In [1]:
import logging

import cicada.logging
import cicada.communicator

logging.basicConfig(level=logging.INFO)

def main(parent):
    log = cicada.logging.Logger(logging.getLogger(), parent)
    
    log.info(f"Player {parent.rank} parent name: {parent.name!r} ranks: {parent.ranks}")
    
cicada.communicator.SocketCommunicator.run(world_size=4, fn=main);

INFO:cicada.communicator.socket:Comm 'world' player 0 rendezvous with tcp://127.0.0.1:56745 from tcp://127.0.0.1:56745.
INFO:cicada.communicator.socket:Comm 'world' player 1 rendezvous with tcp://127.0.0.1:56745 from tcp://127.0.0.1:56746.
INFO:cicada.communicator.socket:Comm 'world' player 2 rendezvous with tcp://127.0.0.1:56745 from tcp://127.0.0.1:56748.
INFO:cicada.communicator.socket:Comm 'world' player 3 rendezvous with tcp://127.0.0.1:56745 from tcp://127.0.0.1:56750.
INFO:cicada.communicator.socket:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 3 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 2 communicator ready.
INFO:root:Player 0 parent name: 'world' ranks: [0, 1, 2, 3]
INFO:root:Player 1 parent name: 'world' ranks: [0, 1, 2, 3]
INFO:root:Player 2 parent name: 'world' ranks: [0, 1, 2, 3]
INFO:root:Player 3 parent name: 'world' rank

In [2]:
def main(parent):
    log = cicada.logging.Logger(logging.getLogger(), parent)
    
    log.info(f"Player {parent.rank} parent name: {parent.name!r} ranks: {parent.ranks}")
    with parent.split(name="child") as child:
        log.info(f"Player {parent.rank} child name: {child.name!r} ranks: {child.ranks}")
    
cicada.communicator.SocketCommunicator.run(world_size=4, fn=main);

INFO:cicada.communicator.socket:Comm 'world' player 0 rendezvous with tcp://127.0.0.1:56777 from tcp://127.0.0.1:56777.
INFO:cicada.communicator.socket:Comm 'world' player 1 rendezvous with tcp://127.0.0.1:56777 from tcp://127.0.0.1:56778.
INFO:cicada.communicator.socket:Comm 'world' player 2 rendezvous with tcp://127.0.0.1:56777 from tcp://127.0.0.1:56780.
INFO:cicada.communicator.socket:Comm 'world' player 3 rendezvous with tcp://127.0.0.1:56777 from tcp://127.0.0.1:56782.
INFO:cicada.communicator.socket:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 2 communicator ready.
INFO:root:Player 0 parent name: 'world' ranks: [0, 1, 2, 3]
INFO:cicada.communicator.socket:Comm 'world' player 3 communicator ready.
INFO:root:Player 1 parent name: 'world' ranks: [0, 1, 2, 3]
INFO:root:Player 2 parent name: 'world' ranks: [0, 1, 2, 3]
INFO:root:Player 3 parent name: 'world' rank

In [3]:
def main(parent):
    log = cicada.logging.Logger(logging.getLogger(), parent)
    
    log.info(f"Player {parent.rank} parent name: {parent.name!r} ranks: {parent.ranks}")
    
    if parent.rank in [0, 1]:
        name = "red"
    if parent.rank in [2, 3]:
        name = "blue"
    
    with parent.split(name=name) as child:
        log.info(f"Player {parent.rank} child name: {child.name!r} ranks: {child.ranks}")
    
cicada.communicator.SocketCommunicator.run(world_size=4, fn=main);

INFO:cicada.communicator.socket:Comm 'world' player 0 rendezvous with tcp://127.0.0.1:56841 from tcp://127.0.0.1:56841.
INFO:cicada.communicator.socket:Comm 'world' player 1 rendezvous with tcp://127.0.0.1:56841 from tcp://127.0.0.1:56842.
INFO:cicada.communicator.socket:Comm 'world' player 2 rendezvous with tcp://127.0.0.1:56841 from tcp://127.0.0.1:56844.
INFO:cicada.communicator.socket:Comm 'world' player 3 rendezvous with tcp://127.0.0.1:56841 from tcp://127.0.0.1:56847.
INFO:cicada.communicator.socket:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 2 communicator ready.
INFO:root:Player 0 parent name: 'world' ranks: [0, 1, 2, 3]
INFO:cicada.communicator.socket:Comm 'world' player 3 communicator ready.
INFO:root:Player 1 parent name: 'world' ranks: [0, 1, 2, 3]
INFO:root:Player 2 parent name: 'world' ranks: [0, 1, 2, 3]
INFO:root:Player 3 parent name: 'world' rank

In [4]:
import collections
import logging

import numpy

Game = collections.namedtuple("Game", ["communicator", "log", "generator"])

def main(communicator):
    # Setup multiple games with separate communicators.
    games = []
    partitions = [[0, 1, 2], [1, 2, 3], [2, 3, 0]]
    for index, partition in enumerate(partitions):
        game_communicator = communicator.split(name=f"game-{index}" if communicator.rank in partition else None)
        if game_communicator is not None:
            game = Game(
                communicator=game_communicator,
                log=cicada.Logger(logging.getLogger(), game_communicator),
                generator=numpy.random.default_rng(),
                )
            games.append(game)

    # Run games in round-robin fashion.
    for i in range(2):
        for game in games:
            value = f"{game.communicator.name} message {i}" if game.communicator.rank == 0 else None
            value = game.communicator.broadcast(src=0, value=value)
            game.log.info(f"{game.communicator.name} player {game.communicator.rank} received broadcast value: {value}")

    # Cleanup games.
    for game in games:
        game.communicator.free()

cicada.communicator.SocketCommunicator.run(world_size=4, fn=main);

INFO:cicada.communicator.socket:Comm 'world' player 0 rendezvous with tcp://127.0.0.1:56899 from tcp://127.0.0.1:56899.
INFO:cicada.communicator.socket:Comm 'world' player 1 rendezvous with tcp://127.0.0.1:56899 from tcp://127.0.0.1:56902.
INFO:cicada.communicator.socket:Comm 'world' player 2 rendezvous with tcp://127.0.0.1:56899 from tcp://127.0.0.1:56904.
INFO:cicada.communicator.socket:Comm 'world' player 3 rendezvous with tcp://127.0.0.1:56899 from tcp://127.0.0.1:56906.
INFO:cicada.communicator.socket:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 2 communicator ready.
INFO:cicada.communicator.socket:Comm 'world' player 3 communicator ready.
INFO:cicada.communicator.socket:Comm 'game-0' player 0 rendezvous with tcp://127.0.0.1:56917 from tcp://127.0.0.1:56917.
INFO:cicada.communicator.socket:Comm 'game-0' player 2 rendezvous with tcp://127.0.0.1:56917 from tcp:/